In [ ]:
import numpy as np
import pandas as pd

## Cost matrix

In [ ]:
#cost_matrix = pd.read_csv('../costs/costMatrixDistance.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixDuration.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixFinancial.csv', index_col=0)
cost_matrix = pd.read_csv('../costs/costMatrixDistance-2020.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixDuration-2020.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixFinancial-2020.csv', index_col=0)

# Convert the entire DataFrame to integers
cost_matrix = cost_matrix.astype(float)

# If the index of the DataFrame needs to be integers (e.g., if the index is non-numeric):
cost_matrix.index = cost_matrix.index.astype(str)

# If the column names need to be integers (if they are non-numeric or string-based):
cost_matrix.columns = cost_matrix.columns.astype(str)

In [ ]:
cost_matrix

In [ ]:
# Number of shops to visit
n = cost_matrix.shape[0]

# Calculate M
M = 0
for i in range(n):
    for j in range(n):
        M += cost_matrix.iloc[i,j]

# M cost for the path i->i
for i in range(n):
    cost_matrix.iloc[i,i] = M + 1

In [ ]:
cost_matrix

## Shop demands

In [ ]:
# Select the shop locations and their demands
#shop_demands = pd.read_csv('../2015_shop_locations.csv').set_index('id')[['stage', 'demand(kg)']].astype(float)
shop_demands = pd.read_csv('../2020_shop_locations.csv').set_index('id')[['stage', 'demand(kg)']].astype(float)

shop_demands = shop_demands.loc[shop_demands['stage'].isin([1, 2, 3, 4]), 'demand(kg)'] # Choose the stages to visit
shops = shop_demands.index
nodes = shops.copy()
nodes = np.append('Depot', nodes)

## Closest neighbour heuristic

In [ ]:
# Start from the storage location, go to the lowest cost shop, then go to the lowest cost shop from there, and so on until all shops are visited

# Create a list of visited shops
visited = ['Depot'] # Start from the storage location

# Create a list of unvisited shops
unvisited = shop_demands.index.to_list()

# Calculate the cost of the route
cost = 0

# Van capacity
van_capacity = 300
remaining_capacity = van_capacity

In [ ]:
active_arcs = []  # List to store paths for all trucks
current_arcs = []  # Current path for the active truck

while len(unvisited) > 0:
    current = 'Depot'  # Start from the storage location
    while len(unvisited) >= 0:

        # Get the costs of the unvisited shops
        costs = cost_matrix.loc[current, unvisited]

        # Filter out shops that exceed the remaining capacity
        feasible_shops = [shop for shop in unvisited if shop_demands[shop] <= remaining_capacity]

        if not feasible_shops:
            # If no feasible shops, end the current truck's route and reset
            next_shop = 'Depot'  # Go back to the storage location
            current_arcs.append((current, next_shop))
            active_arcs.append(current_arcs)
            current_arcs = []  # Reset the path for the next truck
            remaining_capacity = van_capacity  # Reset the capacity for the new truck
            break  # Exit the loop to start with a new truck

        # Get the costs of the feasible shops
        feasible_costs = costs[costs.index.astype(str).isin(feasible_shops)]

        # Get the index of the lowest cost shop
        next_shop = feasible_costs.idxmin()

        # Store the arc between current and next_shop
        current_arcs.append((current, next_shop))

        # Add the lowest cost shop to the visited list
        visited.append(next_shop)

        # Remove the lowest cost shop from the unvisited list
        unvisited.remove(next_shop)

        # Add the cost of the route
        cost += feasible_costs.min()

        # Update the remaining capacity
        remaining_capacity -= shop_demands[next_shop]

        # Move to the next shop
        current = next_shop


# Cost and route

In [ ]:
# Calculate the cost of the route
print('Cost of the route:', cost)
print('Route:', active_arcs)

In [ ]:
def calculate_route_cost(routes, cost_matrix):
    """
    Calculate the total cost of routes by summing arc costs from the cost matrix.

    Parameters:
    - routes: List of routes, where each route is a list of (start, end) tuples
    - cost_matrix: DataFrame of transportation costs

    Returns:
    - Total route cost
    """
    total_cost = 0
    for route in routes:
        route_cost = sum(cost_matrix.loc[start, end] for start, end in route)
        total_cost += route_cost
    return total_cost

In [ ]:
# Calculate cost of routes
original_routes_cost = calculate_route_cost(active_arcs, cost_matrix)

# Print detailed route cost information
print("Number of routes:", len(active_arcs))
print("Total route cost:", original_routes_cost)

In [ ]:
# Flatten the list of arcs

flattened_arcs = [arc for route in active_arcs for arc in route]

flattened_arcs

In [ ]:
# Convert the list of active arcs to a DataFrame
active_arcs_df = pd.DataFrame(flattened_arcs, columns=['From', 'To'])

# Save the DataFrame to a CSV file
active_arcs_df.to_csv('../routings/routing_closest_neighbor_2020_stage_1_Distance.csv', index=False)